# NYC LEAP Hackathon 2026 - Data Access Guide

Complete guide to accessing hackathon datasets from OSN and Hugging Face.

## Available Data Sources

1. **OSN Cloud Storage** (https://nyu1.osn.mghpcc.org)
   - HRRR (High-Resolution Rapid Refresh)
   - ERA5 NYC subset
   - CorrDiff model outputs

2. **Hugging Face Datasets**
   - ClimSim low-res
   - ClimSim low-res-expanded

**Prerequisites:** Run `leap_startup.ipynb` first!

In [ ]:
# Import required packages
import s3fs
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datasets import load_dataset
from huggingface_hub import hf_hub_download, list_repo_files

print("✅ All imports successful!")

## Part 1: OSN Cloud Storage Access

### 1.1 Configure Anonymous S3 Filesystem

In [ ]:
# OSN Configuration for NYC LEAP Hackathon 2026
OSN_ENDPOINT_URL = "https://nyu1.osn.mghpcc.org"
OSN_BUCKET = "leap-pangeo-manual"
HACKATHON_PREFIX = "hackathon-2026"

# Create anonymous S3 filesystem (no credentials needed)
fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': OSN_ENDPOINT_URL}
)

print("=" * 70)
print("OSN CONFIGURATION")
print("=" * 70)
print(f"📍 Endpoint:  {OSN_ENDPOINT_URL}")
print(f"📍 Bucket:    {OSN_BUCKET}")
print(f"📍 Prefix:    {HACKATHON_PREFIX}")
print(f"📍 Full path: s3://{OSN_BUCKET}/{HACKATHON_PREFIX}/")
print("\n✅ Anonymous S3 filesystem created!")

### 1.2 List Top-Level Directories

Expected directories: `hrrr/`, `era5_cds/nyc/`, `corrdiff/`

In [ ]:
# List top-level directories under hackathon-2026/
base_path = f"{OSN_BUCKET}/{HACKATHON_PREFIX}/"

print(f"Listing contents of: s3://{base_path}\n")
print("Available datasets:")
print("=" * 70)

try:
    directories = fs.ls(base_path)
    
    for i, dir_path in enumerate(directories, 1):
        dir_name = dir_path.split('/')[-1]
        print(f"{i}. {dir_name}/")
    
    print(f"\n✅ Found {len(directories)} top-level directories")
    print("\nExpected directories:")
    print("  - hrrr/          (High-Resolution Rapid Refresh data)")
    print("  - era5_cds/nyc/  (ERA5 reanalysis for NYC region)")
    print("  - corrdiff/      (CorrDiff model outputs)")
    
except Exception as e:
    print(f"⚠️ Error: {e}")

### 1.3 Open Zarr Dataset Lazily with xarray

**Lazy loading** means data isn't loaded into memory until you compute or access it.

In [ ]:
# Example: Open a Zarr store from HRRR or ERA5 lazily
print("Code template for lazy loading:")
print("=" * 70)
print(f"""
# Open Zarr store with lazy loading
zarr_path = f"s3://{OSN_BUCKET}/{HACKATHON_PREFIX}/hrrr/your-file.zarr"
mapper = fs.get_mapper(zarr_path)
ds = xr.open_zarr(mapper, consolidated=True)

# Data is NOT loaded into memory yet!
print(ds)  # Shows structure but doesn't load data

# Access a variable (still lazy)
temperature = ds['temperature']

# Compute a subset (NOW data is loaded)
nyc_subset = ds.sel(lat=slice(40.5, 41.0), lon=slice(-74.5, -73.5))
mean_temp = nyc_subset['temperature'].mean().compute()
""")

## Part 2: Hugging Face ClimSim Dataset

### 2.1 ClimSim Dataset Information

In [ ]:
print("=" * 70)
print("CLIMSIM DATASETS ON HUGGING FACE")
print("=" * 70)

datasets_info = [
    {
        'name': 'LEAP/ClimSim_low-res',
        'description': 'Low-resolution climate simulation data'
    },
    {
        'name': 'LEAP/ClimSim_low-res-expanded',
        'description': 'Expanded low-resolution with more variables'
    }
]

for i, dataset in enumerate(datasets_info, 1):
    print(f"\n{i}. {dataset['name']}")
    print(f"   {dataset['description']}")
    print(f"   URL: https://huggingface.co/datasets/{dataset['name']}")

### 2.2 Download Small ClimSim Subsample

In [ ]:
# Download small sample from ClimSim
import os

# Create download directory in scratch space
scratch_dir = Path(f"/home/jovyan/leap-scratch/{os.environ.get('USER', 'default')}")
download_dir = scratch_dir / "climsim_samples"
download_dir.mkdir(parents=True, exist_ok=True)

print(f"Download directory: {download_dir}\n")

repo_id = "LEAP/ClimSim_low-res"

try:
    # Download README first
    print(f"Downloading README from {repo_id}...")
    
    downloaded_path = hf_hub_download(
        repo_id=repo_id,
        filename="README.md",
        repo_type="dataset",
        local_dir=download_dir,
        local_dir_use_symlinks=False
    )
    
    print(f"✅ Downloaded to: {downloaded_path}")
    
except Exception as e:
    print(f"⚠️ Error: {e}")

### 2.3 Load ClimSim with Streamingg

In [ ]:
# Load ClimSim with streaming (doesn't download everything)
repo_id = "LEAP/ClimSim_low-res"

print(f"Loading {repo_id} with streaming...\n")
print("Code template:")
print("=" * 70)
print("""
from datasets import load_dataset

# Streaming mode - access data without full download
dataset = load_dataset(
    'LEAP/ClimSim_low-res',
    split='train',
    streaming=True
)

# Get first few examples
for i, example in enumerate(dataset):
    print(f"Example {i}:", example.keys())
    if i >= 4:
        break
""")

print("\n📝 See the next notebook (02_climsim_analysis.ipynb) for detailed ClimSim inspection!")